**1. This workshop's topic:**
- Split a complex job in to some sub tasks, the sample complex task is to get the real time weather of a city.
- Auto Gen python code to retrieve weather.
- Execute auto generated code as an AI Agent.
- Get the Real weather as LLM cannot get real time data by itself.

**2. Background**
- Whole Picture

<img src="img.png" width="500">

- In this workshop

<img src="img2.png" width="500">

**3. Preparation:**
- Register an account in openweathermap : https://openweathermap.org/
- Get API key at https://home.openweathermap.org/api_keys , hold it. Or you can use mine: **eed0e200d2ce2853033d881a1ff523e7**
- Prepare python environment.

**4. Import needed python libraries.**

In [1]:
import os
from datetime import datetime
from typing import Callable, Dict, Literal, Optional, Union

from typing_extensions import Annotated

from autogen import (
    Agent,
    AssistantAgent,
    ConversableAgent,
    GroupChat,
    GroupChatManager,
    UserProxyAgent,
    config_list_from_json,
    register_function
)
from autogen.agentchat.contrib import agent_builder
from autogen.cache.in_memory_cache import InMemoryCache
from autogen.coding import DockerCommandLineCodeExecutor, LocalCommandLineCodeExecutor

**5. Set the LLM configuration.**

**DO NOT publish this key in public, its mine. :)**

**You can change api_key and base_url to yours.**

In [2]:
os.environ["OPENAI_API_KEY"] = "your-api-key"
os.environ["OPENAI_BASE_URL"] = "your-base-url"
config_list = [
    {"model": "gpt-4o", 
    "api_key": os.environ["OPENAI_API_KEY"],
    "api_type": "azure", 
    "base_url":os.environ["OPENAI_BASE_URL"],
    "api_version":"2024-05-01-preview"},
    {"model": "gpt-4o",
     "api_key": os.environ["OPENAI_API_KEY"], 
     "api_type": "azure",
     "base_url":os.environ["OPENAI_BASE_URL"],
     "api_version":"2024-05-01-preview"},
]

**Create a planner agent.**

**It will act as a planer to split the complex task into pieces, and it will fix the error in each flows. Hope it can do it well. :)**

In [3]:
# Create planner agent.
planner = AssistantAgent(
    name="planner",
    display_name="👨‍🏫(planner)",
    llm_config={
        "config_list": config_list,
        "cache_seed": None,  # Disable legacy cache.
    },
    system_message=""" You are a helpful AI assistant. You suggest a feasible plan
    for finishing a complex task by decomposing it into 3-5 sub-tasks. 
    If the plan is not good, suggest a better plan. 
    If the execution is wrong, analyze the error and suggest a fix.""",
)


# Create a planner user agent used to interact with the planner.
planner_user = UserProxyAgent(
    name="planner_self",
    display_name="👨‍🏫(planner self)",
    human_input_mode="NEVER",
    code_execution_config=False,
)

# The function for asking the planner.

def task_planner(question: Annotated[str, "Question to ask the planner."]) -> str:
    with InMemoryCache() as cache:
        planner_user.initiate_chat(planner, message=question, max_turns=1, cache=cache)
    # return the last message received from the planner
    return planner_user.last_message()["content"]

**Create an assistant agent, it will go to complete the detail sub tasks, include generate python codes.**

In [4]:
# Create assistant agent.
assistant = AssistantAgent(
    name="assistant",
    display_name="👨‍💻(assistant)",
    system_message="""
    You are a helpful AI assistant. You have an API Key of OpenWeatherMap: eed0e200d2ce2853033d881a1ff523e7.
    You need to use the task planner to decompose a complex task into sub-tasks. 
    Make sure your follow through the sub-tasks. 
    If you cannot get the realtime data, write Python code in markdown blocks, and I will execute them.
    Give the user a final solution at the end. 
    Return a word TERMINATE only if the sub-tasks are completed
    """,
    llm_config={
        "config_list": config_list,
        "cache_seed": None,  # Disable legacy cache.
    },
)

# Setting up code executor.
os.makedirs("planning", exist_ok=True)
# Use DockerCommandLineCodeExecutor to run code in a docker container.
# code_executor = DockerCommandLineCodeExecutor(work_dir="planning")
code_executor = LocalCommandLineCodeExecutor(work_dir="planning")

# Create user proxy agent used to interact with the assistant.
user_proxy = UserProxyAgent(
    name="user",
    display_name="👨‍⚖️(user)",
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: "content" in x
    and x["content"] is not None
    and x["content"].rstrip().endswith("TERMINATE"),
    code_execution_config={"executor": code_executor},
)

# Register the function to the agent pair.
register_function(
    task_planner,
    caller=assistant,
    executor=user_proxy,
    name="task_planner",
    description="A task planner than can help you with decomposing a complex task into sub-tasks.",
)

**Define the task of today workshop: Ask it how the weather like of Da Nang today.**

In [5]:
task = (
    f"Today is {datetime.now().date()}. How can I get the weather of a place?"
)
print(task)

Today is 2025-03-20. How can I get the weather of a place?


**Run task.**

When two agents are communicating with each other, we have chance to help them.

Do you remember the API key of openweathermap? In most of situation, AI will generate code based on openweathermap.

But if not, you can type in this requirement at your turn to input.

You can input empty in the textbox, means they are playing well, just continue.

**IF any assistant said that: Error: Function python not found. Please reply nothing and wait assitant generate a new code.**


In [ ]:
# Use Cache.disk to cache LLM responses. Change cache_seed for different responses.
with InMemoryCache() as cache:
    # the assistant receives a message from the user, which contains the task description
    user_proxy.initiate_chat(
        assistant,
        message=task,
        cache=cache
    )

👨‍⚖️(user) (to 👨‍💻(assistant)):

Today is 2025-03-20. How can I get the weather of a place?

--------------------------------------------------------------------------------
👨‍💻(assistant) (to 👨‍⚖️(user)):

***** Suggested tool call (call_rjv2bQInUmfRBzPBeOQpFhi3): task_planner *****
Arguments: 
{"question":"How to get the weather of a place?"}
*****************************************************************************

--------------------------------------------------------------------------------


Replying as user. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION task_planner...
👨‍🏫(planner self) (to 👨‍🏫(planner)):

How to get the weather of a place?

--------------------------------------------------------------------------------
👨‍🏫(planner) (to 👨‍🏫(planner self)):

To get the weather of a place, here is a feasible plan broken down into 3-5 sub-tasks:

### Plan:
1. **Identify the Location**:
   - Determine the place for which you need weather information.
   - Confirm its name and, if necessary, detailed geographical specifics such as city, region, country, or latitude/longitude.

2. **Choose a Weather Information Source**:
   - Select a reliable weather service or platform such as Google Weather, AccuWeather, Weather.com, OpenWeatherMap, or a weather API.

3. **Retrieve Weather Data**:
   - Use the chosen source to look up the weather for the identified location.
   - If using an API, ensure you have access credentials or key, and format the query c

Replying as user. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Da Nang


👨‍⚖️(user) (to 👨‍💻(assistant)):

Da Nang

--------------------------------------------------------------------------------
👨‍💻(assistant) (to 👨‍⚖️(user)):

To get the weather for Da Nang, I will use the OpenWeatherMap API to fetch real-time data. I need to write Python code to retrieve this information since I cannot fetch real-time data directly. Here's the code:

```python
import requests

# API credentials
api_key = "eed0e200d2ce2853033d881a1ff523e7"
city_name = "Da Nang"
base_url = "https://api.openweathermap.org/data/2.5/weather"

# Construct the request URL
params = {
    "q": city_name,
    "appid": api_key,
    "units": "metric"  # Metric units for temperature in Celsius
}

# Fetch the weather data
response = requests.get(base_url, params=params)
weather_data = response.json()

# Display the output
weather_data
```

Let me know once the Python code is executed so I can analyze the weather data for Da Nang!

-----------------------------------------------------------------------

...


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
👨‍⚖️(user) (to 👨‍💻(assistant)):

exitcode: 0 (execution succeeded)
Code output: 

--------------------------------------------------------------------------------
👨‍💻(assistant) (to 👨‍⚖️(user)):

It looks like the execution succeeded, but no output data has been provided. We need the weather data to proceed further. Could you please share the weather data or rerun the code and provide the JSON output received from OpenWeatherMap?

--------------------------------------------------------------------------------


Replying as user. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  change the code and print output


👨‍⚖️(user) (to 👨‍💻(assistant)):

change the code and print output

--------------------------------------------------------------------------------
👨‍💻(assistant) (to 👨‍⚖️(user)):

Here’s the modified code to ensure the response output is printed. Please run it again:

```python
import requests

# API credentials
api_key = "eed0e200d2ce2853033d881a1ff523e7"
city_name = "Da Nang"
base_url = "https://api.openweathermap.org/data/2.5/weather"

# Construct the request URL
params = {
    "q": city_name,
    "appid": api_key,
    "units": "metric"  # Metric units for temperature in Celsius
}

# Fetch the weather data
response = requests.get(base_url, params=params)
weather_data = response.json()

# Print the output
print(weather_data)
```

Once you have the printed output from this modified code, please share it with me so I can interpret the weather information for Da Nang!

--------------------------------------------------------------------------------


Replying as user. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
👨‍⚖️(user) (to 👨‍💻(assistant)):

exitcode: 0 (execution succeeded)
Code output: {'coord': {'lon': 108.2208, 'lat': 16.0678}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 19.99, 'feels_like': 20.21, 'temp_min': 19.99, 'temp_max': 19.99, 'pressure': 1025, 'humidity': 83, 'sea_level': 1025, 'grnd_level': 1020}, 'visibility': 10000, 'wind': {'speed': 1.03, 'deg': 0}, 'clouds': {'all': 75}, 'dt': 1742441292, 'sys': {'type': 1, 'id': 9306, 'country': 'VN', 'sunrise': 1742424686, 'sunset': 1742468272}, 'timezone': 25200, 'id': 1583992, 'name': 'Turan', 'cod': 200}


--------------------------------------------------------------------------------
👨‍💻(assistant) (to 👨‍⚖️(user)):

Here is the interpreted weather data for Da Nang:

- **Location**: Da Nang, Vietnam
- **Coordinates**: Latit